federated and semi supervised using gans + cnn

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import joblib
import copy

# === Step 1: Data Preparation ===
df = pd.read_csv("new_network_train.csv")
X = df.drop(columns=["ProtocolName"]).values
y = df["ProtocolName"].astype(np.int64).values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, "scaler.pkl")

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# === Step 2: Data Partitioning (Federated Clients) ===
NUM_CLIENTS = 5
dataset = TensorDataset(X_train_tensor, y_train_tensor)
total_len = len(dataset)
base_size = total_len // NUM_CLIENTS
remainder = total_len % NUM_CLIENTS

partition_sizes = [base_size + (1 if i < remainder else 0) for i in range(NUM_CLIENTS)]
client_partitions = random_split(dataset, partition_sizes)

# === Step 3: Setup ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_dim = X.shape[1]
num_classes = len(np.unique(y))
latent_dim = 100
label_smooth_real = 0.9

def one_hot(labels, num_classes):
    return torch.eye(num_classes, device=labels.device)[labels]

class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0):
        super().__init__()
        self.gamma = gamma

    def forward(self, input, target):
        logpt = F.log_softmax(input, dim=1)
        pt = torch.exp(logpt)
        loss = (1 - pt) ** self.gamma * logpt
        return F.nll_loss(loss, target)

# === Models ===
class Generator(nn.Module):
    def __init__(self, noise_dim, label_dim, output_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(noise_dim + label_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, output_dim),
        )

    def forward(self, noise, labels):
        labels = one_hot(labels, num_classes).to(noise.device)
        x = torch.cat([noise, labels], dim=1)
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, input_dim, label_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim + label_dim, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, data, labels):
        labels = one_hot(labels, num_classes).to(data.device)
        x = torch.cat([data, labels], dim=1)
        return self.model(x)

class CNNClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.net(x)

def get_confident_pseudo_labels(model, data, threshold=0.9):
    model.eval()
    data = data.to(device)
    with torch.no_grad():
        outputs = model(data)
        probs = F.softmax(outputs, dim=1)
        confidences, predictions = torch.max(probs, dim=1)
        mask = confidences >= threshold
        return data[mask].cpu(), predictions[mask].cpu()

def fed_avg_weighted(models, data_sizes):
    """
    Federated Averaging with weighted contributions.
    """
    avg_model = copy.deepcopy(models[0])
    total_data = sum(data_sizes)

    for key in avg_model.state_dict().keys():
        weighted_sum = sum(
            (model.state_dict()[key].float() * (data_size / total_data))
            for model, data_size in zip(models, data_sizes)
        )
        avg_model.state_dict()[key].copy_(weighted_sum)

    return avg_model

# === Init Global Models ===
global_G = Generator(latent_dim, num_classes, input_dim).to(device)
global_D = Discriminator(input_dim, num_classes).to(device)
global_C = CNNClassifier(input_dim, num_classes).to(device)

bce = nn.BCELoss()
focal_loss = FocalLoss()
scheduler_C = optim.lr_scheduler.StepLR(optim.Adam(global_C.parameters()), step_size=15, gamma=0.5)

# === Step 4: Federated Learning Loop ===
EPOCHS = 50
for epoch in range(EPOCHS):
    local_G_models, local_D_models, local_C_models = [], [], []
    local_data_sizes = []

    total_d_loss = total_g_loss = total_c_loss = 0

    for client_data in client_partitions:
        loader = DataLoader(client_data, batch_size=64, shuffle=True)
        local_data_sizes.append(len(client_data))

        G = copy.deepcopy(global_G).train()
        D = copy.deepcopy(global_D).train()
        C = copy.deepcopy(global_C).train()

        opt_G = optim.Adam(G.parameters(), lr=0.0002)
        opt_D = optim.Adam(D.parameters(), lr=0.0002)
        opt_C = optim.Adam(C.parameters(), lr=0.001)

        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            batch_size = xb.size(0)

            real_labels = torch.full((batch_size, 1), label_smooth_real).to(device)
            fake_labels = torch.zeros((batch_size, 1)).to(device)

            z = torch.randn(batch_size, latent_dim).to(device)
            fake_data = G(z, yb)

            D_real = D(xb, yb)
            D_fake = D(fake_data.detach(), yb)
            loss_D = bce(D_real, real_labels) + bce(D_fake, fake_labels)
            opt_D.zero_grad(); loss_D.backward(); opt_D.step()

            D_fake = D(fake_data, yb)
            loss_G = bce(D_fake, real_labels)
            opt_G.zero_grad(); loss_G.backward(); opt_G.step()

            synthetic_data = G(torch.randn(batch_size, latent_dim).to(device), yb)
            combined_data = torch.cat([xb, synthetic_data], dim=0)
            combined_labels = torch.cat([yb, yb], dim=0)
            preds = C(combined_data)
            loss_C = focal_loss(preds, combined_labels)
            opt_C.zero_grad(); loss_C.backward(); opt_C.step()

            total_d_loss += loss_D.item()
            total_g_loss += loss_G.item()
            total_c_loss += loss_C.item()

        local_G_models.append(G)
        local_D_models.append(D)
        local_C_models.append(C)

    # === Aggregate Models (FedAvg Weighted) ===
    global_G = fed_avg_weighted(local_G_models, local_data_sizes).to(device)
    global_D = fed_avg_weighted(local_D_models, local_data_sizes).to(device)
    global_C = fed_avg_weighted(local_C_models, local_data_sizes).to(device)

    # === Pseudo-label training with confident predictions ===
    pseudo_data, pseudo_labels = get_confident_pseudo_labels(global_C, X_test_tensor)
    if len(pseudo_data) > 0:
        semi_loader = DataLoader(
            TensorDataset(torch.cat([X_train_tensor, pseudo_data]),
                          torch.cat([y_train_tensor, pseudo_labels])),
            batch_size=128, shuffle=True
        )
        global_C.train()
        opt_C = optim.Adam(global_C.parameters(), lr=0.001)
        for xb, yb in semi_loader:
            xb, yb = xb.to(device), yb.to(device)
            preds = global_C(xb)
            loss = focal_loss(preds, yb)
            opt_C.zero_grad(); loss.backward(); opt_C.step()

    scheduler_C.step()
    print(f"[Epoch {epoch+1}] D Loss: {total_d_loss:.2f}, G Loss: {total_g_loss:.2f}, C Loss: {total_c_loss:.2f}")

# === Step 5: Evaluation ===
global_C.eval()
with torch.no_grad():
    preds = global_C(X_test_tensor.to(device))
    y_pred = preds.argmax(dim=1).cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()

label_map = {
    0: 'AMAZON', 1: 'CLOUDFLARE', 2: 'DROPBOX', 3: 'FACEBOOK',
    4: 'GMAIL', 5: 'GOOGLE', 6: 'HTTP', 7: 'HTTP_CONNECT', 8: 'HTTP_PROXY',
    9: 'MICROSOFT', 10: 'MSN', 11: 'SKYPE', 12: 'SSL', 13: 'TWITTER',
    14: 'WINDOWS_UPDATE', 15: 'YAHOO', 16: 'YOUTUBE'
}

print("\n✅ Accuracy:", accuracy_score(y_true, y_pred))
print("\n📊 Classification Report:\n", classification_report(
    y_true, y_pred, target_names=[label_map[i] for i in sorted(label_map)]))

# === Save Final Models ===
torch.save(global_G.state_dict(), "fed_generator_cgan_ssl.pth")
torch.save(global_D.state_dict(), "fed_discriminator_cgan_ssl.pth")
torch.save(global_C.state_dict(), "fed_classifier_cgan_ssl.pth")


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:227: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


[Epoch 1] D Loss: 1345.43, G Loss: 4444.47, C Loss: 3208.47
[Epoch 2] D Loss: 1359.94, G Loss: 4336.36, C Loss: 2890.45
[Epoch 3] D Loss: 1254.65, G Loss: 5016.52, C Loss: 3030.93
[Epoch 4] D Loss: 1340.86, G Loss: 4365.69, C Loss: 2950.19
[Epoch 5] D Loss: 1436.78, G Loss: 3921.60, C Loss: 2918.80
[Epoch 6] D Loss: 1464.32, G Loss: 3581.19, C Loss: 2920.37
[Epoch 7] D Loss: 1399.76, G Loss: 3700.56, C Loss: 2854.87
[Epoch 8] D Loss: 1440.38, G Loss: 3482.52, C Loss: 2862.29
[Epoch 9] D Loss: 1409.12, G Loss: 3461.93, C Loss: 3424.02
[Epoch 10] D Loss: 1400.02, G Loss: 3599.39, C Loss: 2817.09
[Epoch 11] D Loss: 1366.39, G Loss: 3701.98, C Loss: 2783.34
[Epoch 12] D Loss: 1427.20, G Loss: 3394.79, C Loss: 2603.62
[Epoch 13] D Loss: 1482.15, G Loss: 3230.43, C Loss: 2314.94
[Epoch 14] D Loss: 1528.14, G Loss: 3155.00, C Loss: 2035.51
[Epoch 15] D Loss: 1548.14, G Loss: 3025.40, C Loss: 1843.51
[Epoch 16] D Loss: 1548.47, G Loss: 2991.18, C Loss: 1766.90
[Epoch 17] D Loss: 1622.03, G Los

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import joblib
import copy

# === Step 1: Data Preparation ===
df = pd.read_csv("new_network_train.csv")
X = df.drop(columns=["ProtocolName"]).values
y = df["ProtocolName"].astype(np.int64).values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, "scaler.pkl")

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# === Step 2: Data Partitioning (Federated Clients) ===
NUM_CLIENTS = 5
dataset = TensorDataset(X_train_tensor, y_train_tensor)
total_len = len(dataset)
base_size = total_len // NUM_CLIENTS
remainder = total_len % NUM_CLIENTS

partition_sizes = [base_size + (1 if i < remainder else 0) for i in range(NUM_CLIENTS)]
client_partitions = random_split(dataset, partition_sizes)

# === Step 3: Setup ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_dim = X.shape[1]
num_classes = len(np.unique(y))
latent_dim = 100
label_smooth_real = 0.9

def one_hot(labels, num_classes):
    return torch.eye(num_classes, device=labels.device)[labels]

class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0):
        super().__init__()
        self.gamma = gamma

    def forward(self, input, target):
        logpt = F.log_softmax(input, dim=1)
        pt = torch.exp(logpt)
        loss = (1 - pt) ** self.gamma * logpt
        return F.nll_loss(loss, target)

# === Models ===
class Generator(nn.Module):
    def __init__(self, noise_dim, label_dim, output_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(noise_dim + label_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, output_dim),
        )

    def forward(self, noise, labels):
        labels = one_hot(labels, num_classes).to(noise.device)
        x = torch.cat([noise, labels], dim=1)
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, input_dim, label_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim + label_dim, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, data, labels):
        labels = one_hot(labels, num_classes).to(data.device)
        x = torch.cat([data, labels], dim=1)
        return self.model(x)

class CNNClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.net(x)

def get_confident_pseudo_labels(model, data, threshold=0.9):
    model.eval()
    data = data.to(device)
    with torch.no_grad():
        outputs = model(data)
        probs = F.softmax(outputs, dim=1)
        confidences, predictions = torch.max(probs, dim=1)
        mask = confidences >= threshold
        return data[mask].cpu(), predictions[mask].cpu()

def fed_avg_weighted(models, data_sizes):
    avg_model = copy.deepcopy(models[0])
    total_data = sum(data_sizes)
    for key in avg_model.state_dict().keys():
        weighted_sum = sum(
            (model.state_dict()[key].float() * (data_size / total_data))
            for model, data_size in zip(models, data_sizes)
        )
        avg_model.state_dict()[key].copy_(weighted_sum)
    return avg_model

# === Init Global Models ===
global_G = Generator(latent_dim, num_classes, input_dim).to(device)
global_D = Discriminator(input_dim, num_classes).to(device)
global_C = CNNClassifier(input_dim, num_classes).to(device)

bce = nn.BCELoss()
focal_loss = FocalLoss()
scheduler_C = optim.lr_scheduler.StepLR(optim.Adam(global_C.parameters()), step_size=15, gamma=0.5)

# === Step 4: Federated Learning Loop ===
EPOCHS = 50
for epoch in range(EPOCHS):
    print(f"\n=== Epoch {epoch + 1} ===")
    local_G_models, local_D_models, local_C_models = [], [], []
    local_data_sizes = []

    total_d_loss = total_g_loss = total_c_loss = 0

    for client_idx, client_data in enumerate(client_partitions):
        print(f"\n🚀 Training on Client {client_idx + 1}...")

        loader = DataLoader(client_data, batch_size=64, shuffle=True)
        local_data_sizes.append(len(client_data))

        G = copy.deepcopy(global_G).train()
        D = copy.deepcopy(global_D).train()
        C = copy.deepcopy(global_C).train()

        opt_G = optim.Adam(G.parameters(), lr=0.0002)
        opt_D = optim.Adam(D.parameters(), lr=0.0002)
        opt_C = optim.Adam(C.parameters(), lr=0.001)

        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            batch_size = xb.size(0)

            real_labels = torch.full((batch_size, 1), label_smooth_real).to(device)
            fake_labels = torch.zeros((batch_size, 1)).to(device)

            z = torch.randn(batch_size, latent_dim).to(device)
            fake_data = G(z, yb)

            D_real = D(xb, yb)
            D_fake = D(fake_data.detach(), yb)
            loss_D = bce(D_real, real_labels) + bce(D_fake, fake_labels)
            opt_D.zero_grad(); loss_D.backward(); opt_D.step()

            D_fake = D(fake_data, yb)
            loss_G = bce(D_fake, real_labels)
            opt_G.zero_grad(); loss_G.backward(); opt_G.step()

            synthetic_data = G(torch.randn(batch_size, latent_dim).to(device), yb)
            combined_data = torch.cat([xb, synthetic_data], dim=0)
            combined_labels = torch.cat([yb, yb], dim=0)
            preds = C(combined_data)
            loss_C = focal_loss(preds, combined_labels)
            opt_C.zero_grad(); loss_C.backward(); opt_C.step()

            total_d_loss += loss_D.item()
            total_g_loss += loss_G.item()
            total_c_loss += loss_C.item()

        local_G_models.append(G)
        local_D_models.append(D)
        local_C_models.append(C)

    global_G = fed_avg_weighted(local_G_models, local_data_sizes).to(device)
    global_D = fed_avg_weighted(local_D_models, local_data_sizes).to(device)
    global_C = fed_avg_weighted(local_C_models, local_data_sizes).to(device)

    pseudo_data, pseudo_labels = get_confident_pseudo_labels(global_C, X_test_tensor)
    if len(pseudo_data) > 0:
        print(f"📈 Adding {len(pseudo_data)} pseudo-labeled samples to training data.")
        semi_loader = DataLoader(
            TensorDataset(torch.cat([X_train_tensor, pseudo_data]),
                          torch.cat([y_train_tensor, pseudo_labels])),
            batch_size=128, shuffle=True
        )
        global_C.train()
        opt_C = optim.Adam(global_C.parameters(), lr=0.001)
        for xb, yb in semi_loader:
            xb, yb = xb.to(device), yb.to(device)
            preds = global_C(xb)
            loss = focal_loss(preds, yb)
            opt_C.zero_grad(); loss.backward(); opt_C.step()

    scheduler_C.step()
    print(f"[Epoch {epoch+1}] D Loss: {total_d_loss:.2f}, G Loss: {total_g_loss:.2f}, C Loss: {total_c_loss:.2f}")

# === Step 5: Evaluation ===
global_C.eval()
with torch.no_grad():
    preds = global_C(X_test_tensor.to(device))
    y_pred = preds.argmax(dim=1).cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()

label_map = {
    0: 'AMAZON', 1: 'CLOUDFLARE', 2: 'DROPBOX', 3: 'FACEBOOK',
    4: 'GMAIL', 5: 'GOOGLE', 6: 'HTTP', 7: 'HTTP_CONNECT', 8: 'HTTP_PROXY',
    9: 'MICROSOFT', 10: 'MSN', 11: 'SKYPE', 12: 'SSL', 13: 'TWITTER',
    14: 'WINDOWS_UPDATE', 15: 'YAHOO', 16: 'YOUTUBE'
}

print("\n✅ Accuracy:", accuracy_score(y_true, y_pred))
print("\n📊 Classification Report:\n", classification_report(
    y_true, y_pred, target_names=[label_map[i] for i in sorted(label_map)]))

# === Save Final Models ===
torch.save(global_G.state_dict(), "fed_generator_cgan_ssl2.pth")
torch.save(global_D.state_dict(), "fed_discriminator_cgan_ssl2.pth")
torch.save(global_C.state_dict(), "fed_classifier_cgan_ssl2.pth")



=== Epoch 1 ===

🚀 Training on Client 1...

🚀 Training on Client 2...

🚀 Training on Client 3...

🚀 Training on Client 4...

🚀 Training on Client 5...
📈 Adding 2 pseudo-labeled samples to training data.


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:227: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


[Epoch 1] D Loss: 1323.87, G Loss: 4463.53, C Loss: 3204.75

=== Epoch 2 ===

🚀 Training on Client 1...

🚀 Training on Client 2...

🚀 Training on Client 3...

🚀 Training on Client 4...

🚀 Training on Client 5...
📈 Adding 2462 pseudo-labeled samples to training data.
[Epoch 2] D Loss: 1462.45, G Loss: 4170.81, C Loss: 2932.72

=== Epoch 3 ===

🚀 Training on Client 1...

🚀 Training on Client 2...

🚀 Training on Client 3...

🚀 Training on Client 4...

🚀 Training on Client 5...
📈 Adding 3790 pseudo-labeled samples to training data.
[Epoch 3] D Loss: 1364.28, G Loss: 4444.82, C Loss: 2948.95

=== Epoch 4 ===

🚀 Training on Client 1...

🚀 Training on Client 2...

🚀 Training on Client 3...

🚀 Training on Client 4...

🚀 Training on Client 5...
📈 Adding 5052 pseudo-labeled samples to training data.
[Epoch 4] D Loss: 1612.90, G Loss: 3516.49, C Loss: 2979.43

=== Epoch 5 ===

🚀 Training on Client 1...

🚀 Training on Client 2...

🚀 Training on Client 3...

🚀 Training on Client 4...

🚀 Training on

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import joblib
import copy

# === Data Preparation ===
df = pd.read_csv("new_network_train.csv")
X = df.drop(columns=["ProtocolName"]).values
y = df["ProtocolName"].astype(np.int64).values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, "scaler.pkl")

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# === Data Partitioning ===
NUM_CLIENTS = 5
dataset = TensorDataset(X_train_tensor, y_train_tensor)
partition_sizes = [len(dataset) // NUM_CLIENTS + (1 if i < len(dataset) % NUM_CLIENTS else 0) for i in range(NUM_CLIENTS)]
client_partitions = random_split(dataset, partition_sizes)

# === Device and Constants ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_dim = X.shape[1]
num_classes = len(np.unique(y))
latent_dim = 100
label_smooth_real = 0.9

def one_hot(labels, num_classes):
    return torch.eye(num_classes, device=labels.device)[labels]

class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0):
        super().__init__()
        self.gamma = gamma
    def forward(self, input, target):
        logpt = F.log_softmax(input, dim=1)
        pt = torch.exp(logpt)
        loss = (1 - pt) ** self.gamma * logpt
        return F.nll_loss(loss, target)

# === Models ===
class Generator(nn.Module):
    def __init__(self, noise_dim, label_dim, output_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(noise_dim + label_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, output_dim),
        )
    def forward(self, noise, labels):
        labels = one_hot(labels, num_classes).to(noise.device)
        x = torch.cat([noise, labels], dim=1)
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, input_dim, label_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim + label_dim, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    def forward(self, data, labels):
        labels = one_hot(labels, num_classes).to(data.device)
        x = torch.cat([data, labels], dim=1)
        return self.model(x)

class CNNClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes)
        )
    def forward(self, x):
        return self.net(x)

def fed_avg_weighted(models, data_sizes):
    avg_model = copy.deepcopy(models[0])
    total_data = sum(data_sizes)
    for key in avg_model.state_dict().keys():
        weighted_sum = sum(
            (model.state_dict()[key].float() * (data_size / total_data))
            for model, data_size in zip(models, data_sizes)
        )
        avg_model.state_dict()[key].copy_(weighted_sum)
    return avg_model

def get_confident_pseudo_labels(model, data, threshold=0.9):
    model.eval()
    data = data.to(device)
    with torch.no_grad():
        outputs = model(data)
        probs = F.softmax(outputs, dim=1)
        confidences, predictions = torch.max(probs, dim=1)
        mask = confidences >= threshold
        return data[mask].cpu(), predictions[mask].cpu()

def train_client(client_data, global_G, global_D, global_C):
    G = copy.deepcopy(global_G).train()
    D = copy.deepcopy(global_D).train()
    C = copy.deepcopy(global_C).train()

    loader = DataLoader(client_data, batch_size=64, shuffle=True)
    opt_G = optim.Adam(G.parameters(), lr=0.0002)
    opt_D = optim.Adam(D.parameters(), lr=0.0002)
    opt_C = optim.Adam(C.parameters(), lr=0.001)

    bce = nn.BCELoss()
    focal_loss = FocalLoss()

    total_d_loss = total_g_loss = total_c_loss = 0

    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        batch_size = xb.size(0)

        real_labels = torch.full((batch_size, 1), label_smooth_real).to(device)
        fake_labels = torch.zeros((batch_size, 1)).to(device)

        z = torch.randn(batch_size, latent_dim).to(device)
        fake_data = G(z, yb)

        D_real = D(xb, yb)
        D_fake = D(fake_data.detach(), yb)
        loss_D = bce(D_real, real_labels) + bce(D_fake, fake_labels)
        opt_D.zero_grad(); loss_D.backward(); opt_D.step()

        D_fake = D(fake_data, yb)
        loss_G = bce(D_fake, real_labels)
        opt_G.zero_grad(); loss_G.backward(); opt_G.step()

        synthetic_data = G(torch.randn(batch_size, latent_dim).to(device), yb)
        combined_data = torch.cat([xb, synthetic_data], dim=0)
        combined_labels = torch.cat([yb, yb], dim=0)
        preds = C(combined_data)
        loss_C = focal_loss(preds, combined_labels)
        opt_C.zero_grad(); loss_C.backward(); opt_C.step()

        total_d_loss += loss_D.item()
        total_g_loss += loss_G.item()
        total_c_loss += loss_C.item()

    return G, D, C, total_d_loss, total_g_loss, total_c_loss

# === Init Global Models ===
global_G = Generator(latent_dim, num_classes, input_dim).to(device)
global_D = Discriminator(input_dim, num_classes).to(device)
global_C = CNNClassifier(input_dim, num_classes).to(device)
scheduler_C = optim.lr_scheduler.StepLR(optim.Adam(global_C.parameters()), step_size=15, gamma=0.5)

# === Federated Learning Loop ===
EPOCHS = 30
for epoch in range(EPOCHS):
    print(f"\n=== Epoch {epoch + 1} ===")
    local_G_models, local_D_models, local_C_models = [], [], []
    local_data_sizes = []
    total_d_loss = total_g_loss = total_c_loss = 0

    for client_idx, client_data in enumerate(client_partitions):
        print(f"\n🚀 Training on Client {client_idx + 1}...")
        G, D, C, d_loss, g_loss, c_loss = train_client(client_data, global_G, global_D, global_C)

        local_G_models.append(G)
        local_D_models.append(D)
        local_C_models.append(C)
        local_data_sizes.append(len(client_data))

        total_d_loss += d_loss
        total_g_loss += g_loss
        total_c_loss += c_loss

    global_G = fed_avg_weighted(local_G_models, local_data_sizes).to(device)
    global_D = fed_avg_weighted(local_D_models, local_data_sizes).to(device)
    global_C = fed_avg_weighted(local_C_models, local_data_sizes).to(device)

    pseudo_data, pseudo_labels = get_confident_pseudo_labels(global_C, X_test_tensor)
    if len(pseudo_data) > 1:
        print(f"📈 Adding {len(pseudo_data)} pseudo-labeled samples to training data.")
        semi_loader = DataLoader(
            TensorDataset(torch.cat([X_train_tensor, pseudo_data]),
                          torch.cat([y_train_tensor, pseudo_labels])),
            batch_size=128, shuffle=True
        )
        global_C.train()
        opt_C = optim.Adam(global_C.parameters(), lr=0.001)
        for xb, yb in semi_loader:
            if xb.shape[0] < 2: continue  # avoid batch norm error
            xb, yb = xb.to(device), yb.to(device)
            preds = global_C(xb)
            loss = FocalLoss()(preds, yb)
            opt_C.zero_grad(); loss.backward(); opt_C.step()

    scheduler_C.step()
    print(f"[Epoch {epoch + 1}] D Loss: {total_d_loss:.2f}, G Loss: {total_g_loss:.2f}, C Loss: {total_c_loss:.2f}")

# === Evaluation ===
global_C.eval()
with torch.no_grad():
    preds = global_C(X_test_tensor.to(device))
    y_pred = preds.argmax(dim=1).cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()

label_map = {
    0: 'AMAZON', 1: 'CLOUDFLARE', 2: 'DROPBOX', 3: 'FACEBOOK',
    4: 'GMAIL', 5: 'GOOGLE', 6: 'HTTP', 7: 'HTTP_CONNECT', 8: 'HTTP_PROXY',
    9: 'MICROSOFT', 10: 'MSN', 11: 'SKYPE', 12: 'SSL', 13: 'TWITTER',
    14: 'WINDOWS_UPDATE', 15: 'YAHOO', 16: 'YOUTUBE'
}

print("\n✅ Accuracy:", accuracy_score(y_true, y_pred))
print("\n📊 Classification Report:\n", classification_report(
    y_true, y_pred, target_names=[label_map[i] for i in sorted(label_map)]))

# === Save Models ===
torch.save(global_G.state_dict(), "fed_generator_cgan_ssl3.pth")
torch.save(global_D.state_dict(), "fed_discriminator_cgan_ssl3.pth")
torch.save(global_C.state_dict(), "fed_classifier_cgan_ssl3.pth")



=== Epoch 1 ===

🚀 Training on Client 1...

🚀 Training on Client 2...

🚀 Training on Client 3...

🚀 Training on Client 4...

🚀 Training on Client 5...
📈 Adding 2 pseudo-labeled samples to training data.


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:227: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


[Epoch 1] D Loss: 1345.19, G Loss: 4465.71, C Loss: 3225.83

=== Epoch 2 ===

🚀 Training on Client 1...

🚀 Training on Client 2...

🚀 Training on Client 3...

🚀 Training on Client 4...

🚀 Training on Client 5...
📈 Adding 2619 pseudo-labeled samples to training data.
[Epoch 2] D Loss: 1534.75, G Loss: 3801.54, C Loss: 2967.33

=== Epoch 3 ===

🚀 Training on Client 1...

🚀 Training on Client 2...

🚀 Training on Client 3...

🚀 Training on Client 4...

🚀 Training on Client 5...
📈 Adding 4020 pseudo-labeled samples to training data.
[Epoch 3] D Loss: 1280.43, G Loss: 4617.27, C Loss: 2859.16

=== Epoch 4 ===

🚀 Training on Client 1...

🚀 Training on Client 2...

🚀 Training on Client 3...

🚀 Training on Client 4...

🚀 Training on Client 5...
📈 Adding 4450 pseudo-labeled samples to training data.
[Epoch 4] D Loss: 1407.30, G Loss: 4219.93, C Loss: 2971.93

=== Epoch 5 ===

🚀 Training on Client 1...

🚀 Training on Client 2...

🚀 Training on Client 3...

🚀 Training on Client 4...

🚀 Training on

fgan1 with entropy and confidence

In [ ]:
# === FGAN-I Semi-Supervised Federated Learning with Entropy + Confidence Filtering ===
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import joblib
import copy

# === Load and Preprocess Dataset ===
df = pd.read_csv("new_network_train.csv")
X = df.drop(columns=["ProtocolName"]).values
y = df["ProtocolName"].astype(np.int64).values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, "scaler_fgani.pkl")

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# === Constants ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_dim = X.shape[1]
num_classes = len(np.unique(y))
latent_dim = 100
NUM_CLIENTS = 5
label_smooth_real = 0.9

# === Partition Data ===
dataset = TensorDataset(X_train_tensor, y_train_tensor)
partition_sizes = [len(dataset) // NUM_CLIENTS + (1 if i < len(dataset) % NUM_CLIENTS else 0) for i in range(NUM_CLIENTS)]
client_partitions = random_split(dataset, partition_sizes)

# === Utility Functions ===
def one_hot(labels, num_classes):
    return torch.eye(num_classes, device=labels.device)[labels]

def get_confident_entropy_pseudo_labels(model, data, conf_thresh=0.9, entropy_thresh=0.5):
    model.eval()
    data = data.to(device)
    with torch.no_grad():
        outputs = model(data)
        probs = F.softmax(outputs, dim=1)
        confidence, preds = probs.max(dim=1)
        entropy = -torch.sum(probs * probs.log(), dim=1)
        mask = (confidence >= conf_thresh) & (entropy <= entropy_thresh)
        return data[mask].cpu(), preds[mask].cpu()

class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0):
        super().__init__()
        self.gamma = gamma
    def forward(self, input, target):
        logpt = F.log_softmax(input, dim=1)
        pt = torch.exp(logpt)
        loss = (1 - pt) ** self.gamma * logpt
        return F.nll_loss(loss, target)

# === Models ===
class Generator(nn.Module):
    def __init__(self, noise_dim, label_dim, output_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(noise_dim + label_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, output_dim),
        )
    def forward(self, noise, labels):
        labels = one_hot(labels, num_classes).to(noise.device)
        x = torch.cat([noise, labels], dim=1)
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, input_dim, label_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim + label_dim, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    def forward(self, data, labels):
        labels = one_hot(labels, num_classes).to(data.device)
        x = torch.cat([data, labels], dim=1)
        return self.model(x)

class CNNClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes)
        )
    def forward(self, x):
        return self.net(x)

# === Training Functions ===
def train_fgani_client(client_data, global_G):
    D = Discriminator(input_dim, num_classes).to(device).train()
    loader = DataLoader(client_data, batch_size=64, shuffle=True)
    opt_D = optim.Adam(D.parameters(), lr=0.0002)
    bce = nn.BCELoss()

    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        batch_size = xb.size(0)
        real_labels = torch.full((batch_size, 1), label_smooth_real).to(device)
        fake_labels = torch.zeros((batch_size, 1)).to(device)
        z = torch.randn(batch_size, latent_dim).to(device)
        fake_data = global_G(z, yb)
        D_real = D(xb, yb)
        D_fake = D(fake_data.detach(), yb)
        loss_D = bce(D_real, real_labels) + bce(D_fake, fake_labels)
        opt_D.zero_grad(); loss_D.backward(); opt_D.step()
    return D

def update_global_generator(global_G, client_discriminators):
    global_G.train()
    opt_G = optim.Adam(global_G.parameters(), lr=0.0002)
    bce = nn.BCELoss()
    for D in client_discriminators:
        D.eval()
        labels = torch.randint(0, num_classes, (64,), device=device)
        z = torch.randn(64, latent_dim, device=device)
        fake_data = global_G(z, labels)
        output = D(fake_data, labels)
        loss = bce(output, torch.full_like(output, label_smooth_real))
        opt_G.zero_grad(); loss.backward(); opt_G.step()

# === Training Loop ===
global_G = Generator(latent_dim, num_classes, input_dim).to(device)
global_C = CNNClassifier(input_dim, num_classes).to(device)
scheduler_C = optim.lr_scheduler.StepLR(optim.Adam(global_C.parameters()), step_size=15, gamma=0.5)
focal_loss = FocalLoss()

EPOCHS = 30
for epoch in range(EPOCHS):
    print(f"\n=== Epoch {epoch + 1} ===")
    local_D_models = []

    for client_idx, client_data in enumerate(client_partitions):
        D = train_fgani_client(client_data, global_G)
        local_D_models.append(D)

    update_global_generator(global_G, local_D_models)

    global_C.train()
    opt_C = optim.Adam(global_C.parameters(), lr=0.001)
    all_data, all_labels = [], []

    for client_data in client_partitions:
        loader = DataLoader(client_data, batch_size=64, shuffle=True)
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            z = torch.randn(xb.size(0), latent_dim).to(device)
            fake_data = global_G(z, yb).detach()
            combined_x = torch.cat([xb, fake_data], dim=0)
            combined_y = torch.cat([yb, yb], dim=0)
            all_data.append(combined_x)
            all_labels.append(combined_y)

    train_loader = DataLoader(TensorDataset(torch.cat(all_data), torch.cat(all_labels)), batch_size=128, shuffle=True)
    for xb, yb in train_loader:
        if xb.shape[0] < 2: continue
        xb, yb = xb.to(device), yb.to(device)
        preds = global_C(xb)
        loss = focal_loss(preds, yb)
        opt_C.zero_grad(); loss.backward(); opt_C.step()
    scheduler_C.step()

    # === Pseudo-Labeling with Entropy + Confidence ===
    pseudo_data, pseudo_labels = get_confident_entropy_pseudo_labels(global_C, X_test_tensor)
    if len(pseudo_data) > 1:
        print(f"Adding {len(pseudo_data)} pseudo-labeled samples to training.")
        semi_loader = DataLoader(
            TensorDataset(torch.cat([X_train_tensor, pseudo_data]), torch.cat([y_train_tensor, pseudo_labels])),
            batch_size=128, shuffle=True
        )
        opt_C_pseudo = optim.Adam(global_C.parameters(), lr=0.001)
        for xb, yb in semi_loader:
            if xb.shape[0] < 2: continue
            xb = xb.clone().detach().to(device)
            yb = yb.clone().detach().to(device)
            preds = global_C(xb)
            loss = focal_loss(preds, yb)
            opt_C_pseudo.zero_grad(); loss.backward(); opt_C_pseudo.step()

# === Evaluation ===
global_C.eval()
with torch.no_grad():
    preds = global_C(X_test_tensor.to(device))
    y_pred = preds.argmax(dim=1).cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()

print("\n✅ Accuracy:", accuracy_score(y_true, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_true, y_pred))

# === Save Models ===
torch.save(global_G.state_dict(), "fgan1_generator.pth")
torch.save(global_C.state_dict(), "fgan1_classifier.pth")



=== Epoch 1 ===


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:227: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Adding 2448 pseudo-labeled samples to training.

=== Epoch 2 ===
Adding 5181 pseudo-labeled samples to training.

=== Epoch 3 ===
Adding 6224 pseudo-labeled samples to training.

=== Epoch 4 ===
Adding 6567 pseudo-labeled samples to training.

=== Epoch 5 ===
Adding 7436 pseudo-labeled samples to training.

=== Epoch 6 ===
Adding 6670 pseudo-labeled samples to training.

=== Epoch 7 ===
Adding 7264 pseudo-labeled samples to training.

=== Epoch 8 ===
Adding 6960 pseudo-labeled samples to training.

=== Epoch 9 ===
Adding 7498 pseudo-labeled samples to training.

=== Epoch 10 ===
Adding 7030 pseudo-labeled samples to training.

=== Epoch 11 ===
Adding 6959 pseudo-labeled samples to training.

=== Epoch 12 ===
Adding 7451 pseudo-labeled samples to training.

=== Epoch 13 ===
Adding 7413 pseudo-labeled samples to training.

=== Epoch 14 ===
Adding 7527 pseudo-labeled samples to training.

=== Epoch 15 ===
Adding 7656 pseudo-labeled samples to training.

=== Epoch 16 ===
Adding 6982 pseudo

In [ ]:
import pandas as pd
import torch
import torch.nn.functional as F
import joblib
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# === Define Classifier Architecture (must match trained model) ===
class CNNClassifier(torch.nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(input_dim, 512),
            torch.nn.BatchNorm1d(512),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(512, 256),
            torch.nn.BatchNorm1d(256),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(256, 128),
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.net(x)

# === Paths & Config ===
input_dim = 69         # Set this based on your dataset
num_classes = 17        # Total classes in ProtocolName
model_path = "fgan1_classifier.pth"
scaler_path = "scaler_fgani.pkl"
test_file_path = "new_network_validation.csv"   # <- Replace with your actual file

# === Load Test Data ===
df = pd.read_csv(test_file_path)

# Optional: Check if true labels exist
has_labels = "ProtocolName" in df.columns

if has_labels:
    y_true = df["ProtocolName"].astype(np.int64).values
    X_test = df.drop(columns=["ProtocolName"]).values
else:
    X_test = df.values
    y_true = None

# === Load Scaler and Normalize ===
scaler = joblib.load(scaler_path)
X_scaled = scaler.transform(X_test)

# === Convert to Torch Tensor ===
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)

# === Load Trained Classifier ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNClassifier(input_dim, num_classes)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device).eval()

# === Predict ===
with torch.no_grad():
    outputs = model(X_tensor.to(device))
    probs = F.softmax(outputs, dim=1)
    predictions = torch.argmax(probs, dim=1).cpu().numpy()

# === Optional: Class Labels Mapping ===
label_map = {
    0: 'AMAZON', 1: 'CLOUDFLARE', 2: 'DROPBOX', 3: 'FACEBOOK',
    4: 'GMAIL', 5: 'GOOGLE', 6: 'HTTP', 7: 'HTTP_CONNECT', 8: 'HTTP_PROXY',
    9: 'MICROSOFT', 10: 'MSN', 11: 'SKYPE', 12: 'SSL', 13: 'TWITTER',
    14: 'WINDOWS_UPDATE', 15: 'YAHOO', 16: 'YOUTUBE'
}
predicted_labels = [label_map[p] for p in predictions]

# === Print Results ===
print("\n📌 Predictions (first 10):")
for i in range(min(10, len(predicted_labels))):
    print(f"Sample {i + 1}: {predicted_labels[i]}")

# === Print Accuracy and Report (if true labels exist) ===
if y_true is not None:
    print("\n✅ Accuracy:", accuracy_score(y_true, predictions))
    print("\n📊 Classification Report:\n", classification_report(
        y_true, predictions, target_names=[label_map[i] for i in sorted(label_map)]
    ))

# === Save Predictions to CSV ===
df_out = pd.DataFrame({
    "Predicted Class": predicted_labels
})
if has_labels:
    df_out["True Label"] = [label_map[y] for y in y_true]

df_out.to_csv("predicted_results.csv", index=False)
print("\n📁 Results saved to predicted_results.csv")



📌 Predictions (first 10):
Sample 1: CLOUDFLARE
Sample 2: SKYPE
Sample 3: MSN
Sample 4: TWITTER
Sample 5: CLOUDFLARE
Sample 6: CLOUDFLARE
Sample 7: CLOUDFLARE
Sample 8: CLOUDFLARE
Sample 9: SKYPE
Sample 10: YAHOO

✅ Accuracy: 0.9958110516934047

📊 Classification Report:
                 precision    recall  f1-score   support

        AMAZON       1.00      1.00      1.00      1980
    CLOUDFLARE       1.00      1.00      1.00      1980
       DROPBOX       0.99      1.00      0.99      1980
      FACEBOOK       1.00      0.99      1.00      1980
         GMAIL       0.99      0.99      0.99      1980
        GOOGLE       0.99      1.00      1.00      1980
          HTTP       1.00      1.00      1.00      1980
  HTTP_CONNECT       1.00      0.99      1.00      1980
    HTTP_PROXY       0.99      1.00      1.00      1980
     MICROSOFT       1.00      1.00      1.00      1980
           MSN       0.99      1.00      0.99      1980
         SKYPE       0.99      0.99      0.99      1980

fgan1 with DEC

In [ ]:
# === FGAN-I Semi-Supervised Federated Learning with DEC Clustering ===
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.cluster import KMeans
import joblib
import copy

# === Load and Preprocess Dataset ===
df = pd.read_csv("new_network_train.csv")
X = df.drop(columns=["ProtocolName"]).values
y = df["ProtocolName"].astype(np.int64).values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, "scaler_fgani.pkl")

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# === Constants ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_dim = X.shape[1]
num_classes = len(np.unique(y))
latent_dim = 100
NUM_CLIENTS = 5
label_smooth_real = 0.9

# === Partition Data ===
dataset = TensorDataset(X_train_tensor, y_train_tensor)
partition_sizes = [len(dataset) // NUM_CLIENTS + (1 if i < len(dataset) % NUM_CLIENTS else 0) for i in range(NUM_CLIENTS)]
client_partitions = random_split(dataset, partition_sizes)

# === Utility Functions ===
def one_hot(labels, num_classes):
    return torch.eye(num_classes, device=labels.device)[labels]

def get_pseudo_labels_via_dec(model, data, k=num_classes):
    model.eval()
    data = data.to(device)
    with torch.no_grad():
        features = model.net[:-1](data).cpu().numpy()  # Extract features before final layer
    kmeans = KMeans(n_clusters=k, random_state=0).fit(features)
    pseudo_data = data.cpu()
    pseudo_labels = torch.tensor(kmeans.labels_, dtype=torch.long)
    return pseudo_data, pseudo_labels

class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0):
        super().__init__()
        self.gamma = gamma
    def forward(self, input, target):
        logpt = F.log_softmax(input, dim=1)
        pt = torch.exp(logpt)
        loss = (1 - pt) ** self.gamma * logpt
        return F.nll_loss(loss, target)

# === Models ===
class Generator(nn.Module):
    def __init__(self, noise_dim, label_dim, output_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(noise_dim + label_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, output_dim),
        )
    def forward(self, noise, labels):
        labels = one_hot(labels, num_classes).to(noise.device)
        x = torch.cat([noise, labels], dim=1)
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, input_dim, label_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim + label_dim, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    def forward(self, data, labels):
        labels = one_hot(labels, num_classes).to(data.device)
        x = torch.cat([data, labels], dim=1)
        return self.model(x)

class CNNClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes)
        )
    def forward(self, x):
        return self.net(x)

# === Training Functions ===
def train_fgani_client(client_data, global_G):
    D = Discriminator(input_dim, num_classes).to(device).train()
    loader = DataLoader(client_data, batch_size=64, shuffle=True)
    opt_D = optim.Adam(D.parameters(), lr=0.0002)
    bce = nn.BCELoss()

    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        batch_size = xb.size(0)
        real_labels = torch.full((batch_size, 1), label_smooth_real).to(device)
        fake_labels = torch.zeros((batch_size, 1)).to(device)
        z = torch.randn(batch_size, latent_dim).to(device)
        fake_data = global_G(z, yb)
        D_real = D(xb, yb)
        D_fake = D(fake_data.detach(), yb)
        loss_D = bce(D_real, real_labels) + bce(D_fake, fake_labels)
        opt_D.zero_grad(); loss_D.backward(); opt_D.step()
    return D

def update_global_generator(global_G, client_discriminators):
    global_G.train()
    opt_G = optim.Adam(global_G.parameters(), lr=0.0002)
    bce = nn.BCELoss()
    for D in client_discriminators:
        D.eval()
        labels = torch.randint(0, num_classes, (64,), device=device)
        z = torch.randn(64, latent_dim, device=device)
        fake_data = global_G(z, labels)
        output = D(fake_data, labels)
        loss = bce(output, torch.full_like(output, label_smooth_real))
        opt_G.zero_grad(); loss.backward(); opt_G.step()

# === Training Loop ===
global_G = Generator(latent_dim, num_classes, input_dim).to(device)
global_C = CNNClassifier(input_dim, num_classes).to(device)
scheduler_C = optim.lr_scheduler.StepLR(optim.Adam(global_C.parameters()), step_size=15, gamma=0.5)
focal_loss = FocalLoss()

EPOCHS = 30
for epoch in range(EPOCHS):
    print(f"\n=== Epoch {epoch + 1} ===")
    local_D_models = []

    for client_idx, client_data in enumerate(client_partitions):
        D = train_fgani_client(client_data, global_G)
        local_D_models.append(D)

    update_global_generator(global_G, local_D_models)

    global_C.train()
    opt_C = optim.Adam(global_C.parameters(), lr=0.001)
    all_data, all_labels = [], []

    for client_data in client_partitions:
        loader = DataLoader(client_data, batch_size=64, shuffle=True)
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            z = torch.randn(xb.size(0), latent_dim).to(device)
            fake_data = global_G(z, yb).detach()
            combined_x = torch.cat([xb, fake_data], dim=0)
            combined_y = torch.cat([yb, yb], dim=0)
            all_data.append(combined_x)
            all_labels.append(combined_y)

    train_loader = DataLoader(TensorDataset(torch.cat(all_data), torch.cat(all_labels)), batch_size=128, shuffle=True)
    for xb, yb in train_loader:
        if xb.shape[0] < 2: continue
        xb, yb = xb.to(device), yb.to(device)
        preds = global_C(xb)
        loss = focal_loss(preds, yb)
        opt_C.zero_grad(); loss.backward(); opt_C.step()
    scheduler_C.step()

    # === Pseudo-Labeling with DEC  ===
    pseudo_data, pseudo_labels = get_pseudo_labels_via_dec(global_C, X_test_tensor)
    if len(pseudo_data) > 1:
        print(f"Adding {len(pseudo_data)} pseudo-labeled samples to training.")
        semi_loader = DataLoader(
            TensorDataset(torch.cat([X_train_tensor, pseudo_data]), torch.cat([y_train_tensor, pseudo_labels])),
            batch_size=128, shuffle=True
        )
        opt_C_pseudo = optim.Adam(global_C.parameters(), lr=0.001)
        for xb, yb in semi_loader:
            if xb.shape[0] < 2: continue
            xb = xb.clone().detach().to(device)
            yb = yb.clone().detach().to(device)
            preds = global_C(xb)
            loss = focal_loss(preds, yb)
            opt_C_pseudo.zero_grad(); loss.backward(); opt_C_pseudo.step()

# === Evaluation ===
global_C.eval()
with torch.no_grad():
    preds = global_C(X_test_tensor.to(device))
    y_pred = preds.argmax(dim=1).cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()

print("\n✅ Accuracy:", accuracy_score(y_true, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_true, y_pred))

# === Save Models ===
torch.save(global_G.state_dict(), "fgan1_generator.pth")
torch.save(global_C.state_dict(), "fgan1_classifier.pth")



=== Epoch 1 ===


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:227: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Adding 26928 pseudo-labeled samples to training.

=== Epoch 2 ===
Adding 26928 pseudo-labeled samples to training.

=== Epoch 3 ===
Adding 26928 pseudo-labeled samples to training.

=== Epoch 4 ===
Adding 26928 pseudo-labeled samples to training.

=== Epoch 5 ===
Adding 26928 pseudo-labeled samples to training.

=== Epoch 6 ===
Adding 26928 pseudo-labeled samples to training.

=== Epoch 7 ===
Adding 26928 pseudo-labeled samples to training.

=== Epoch 8 ===
Adding 26928 pseudo-labeled samples to training.

=== Epoch 9 ===
Adding 26928 pseudo-labeled samples to training.

=== Epoch 10 ===
Adding 26928 pseudo-labeled samples to training.

=== Epoch 11 ===
Adding 26928 pseudo-labeled samples to training.

=== Epoch 12 ===
Adding 26928 pseudo-labeled samples to training.

=== Epoch 13 ===
Adding 26928 pseudo-labeled samples to training.

=== Epoch 14 ===
Adding 26928 pseudo-labeled samples to training.

=== Epoch 15 ===
Adding 26928 pseudo-labeled samples to training.

=== Epoch 16 ===
Add

fgan2 with entropy and confidence

In [ ]:
# === FGAN-II with Feature Matching Loss and Federated Generator/Discriminator ===
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import joblib
import copy

# === Load and Preprocess Dataset ===
df = pd.read_csv("new_network_train.csv")
X = df.drop(columns=["ProtocolName"]).values
y = df["ProtocolName"].astype(np.int64).values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, "scaler_fgani.pkl")

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# === Constants ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_dim = X.shape[1]
num_classes = len(np.unique(y))
latent_dim = 100
NUM_CLIENTS = 5
label_smooth_real = 0.9

# === Partition Data ===
dataset = TensorDataset(X_train_tensor, y_train_tensor)
partition_sizes = [len(dataset) // NUM_CLIENTS + (1 if i < len(dataset) % NUM_CLIENTS else 0) for i in range(NUM_CLIENTS)]
client_partitions = random_split(dataset, partition_sizes)

# === Utility Functions ===
def one_hot(labels, num_classes):
    return torch.eye(num_classes, device=labels.device)[labels]

def get_confident_entropy_pseudo_labels(model, data, conf_thresh=0.9, entropy_thresh=0.5):
    model.eval()
    data = data.to(device)
    with torch.no_grad():
        outputs = model(data)
        probs = F.softmax(outputs, dim=1)
        confidence, preds = probs.max(dim=1)
        entropy = -torch.sum(probs * probs.log(), dim=1)
        mask = (confidence >= conf_thresh) & (entropy <= entropy_thresh)
        return data[mask].cpu(), preds[mask].cpu()

class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0):
        super().__init__()
        self.gamma = gamma
    def forward(self, input, target):
        logpt = F.log_softmax(input, dim=1)
        pt = torch.exp(logpt)
        loss = (1 - pt) ** self.gamma * logpt
        return F.nll_loss(loss, target)

# === Models ===
class Generator(nn.Module):
    def __init__(self, noise_dim, label_dim, output_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(noise_dim + label_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, output_dim),
        )
    def forward(self, noise, labels):
        labels = one_hot(labels, num_classes).to(noise.device)
        x = torch.cat([noise, labels], dim=1)
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, input_dim, label_dim):
        super().__init__()
        self.feature = nn.Sequential(
            nn.Linear(input_dim + label_dim, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    def forward(self, data, labels):
        labels = one_hot(labels, num_classes).to(data.device)
        x = torch.cat([data, labels], dim=1)
        features = self.feature(x)
        return self.classifier(features), features

class CNNClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes)
        )
    def forward(self, x):
        return self.net(x)

# === Federated Functions ===
def federated_average(models, sizes):
    avg_model = copy.deepcopy(models[0])
    total = sum(sizes)
    for key in avg_model.state_dict():
        weighted_sum = sum(model.state_dict()[key] * (size / total) for model, size in zip(models, sizes))
        avg_model.state_dict()[key].copy_(weighted_sum)
    return avg_model

# === Training Functions ===
def train_fgan2_client(client_data, global_G, global_D):
    G = copy.deepcopy(global_G).train()
    D = copy.deepcopy(global_D).train()
    loader = DataLoader(client_data, batch_size=64, shuffle=True)
    opt_D = optim.Adam(D.parameters(), lr=0.0001)
    opt_G = optim.Adam(G.parameters(), lr=0.0002)
    bce = nn.BCELoss()

    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        batch_size = xb.size(0)
        real_labels = torch.full((batch_size, 1), label_smooth_real).to(device)
        fake_labels = torch.zeros((batch_size, 1)).to(device)

        # === Train Discriminator ===
        z = torch.randn(batch_size, latent_dim).to(device)
        fake_data = G(z, yb)
        D_real_out, _ = D(xb, yb)
        D_fake_out, _ = D(fake_data.detach(), yb)
        loss_D = bce(D_real_out, real_labels) + bce(D_fake_out, fake_labels)
        opt_D.zero_grad(); loss_D.backward(); opt_D.step()

        # === Train Generator with Feature Matching ===
        _, real_features = D(xb, yb)
        _, fake_features = D(fake_data, yb)
        loss_G = F.mse_loss(fake_features.mean(dim=0), real_features.mean(dim=0))
        opt_G.zero_grad(); loss_G.backward(); opt_G.step()

    return G, D

# === Training Loop ===
global_G = Generator(latent_dim, num_classes, input_dim).to(device)
global_D = Discriminator(input_dim, num_classes).to(device)
global_C = CNNClassifier(input_dim, num_classes).to(device)
scheduler_C = optim.lr_scheduler.StepLR(optim.Adam(global_C.parameters()), step_size=15, gamma=0.5)
focal_loss = FocalLoss()

EPOCHS = 30
for epoch in range(EPOCHS):
    print(f"\n=== Epoch {epoch + 1} ===")
    local_G, local_D, sizes = [], [], []

    for client_data in client_partitions:
        G_new, D_new = train_fgan2_client(client_data, global_G, global_D)
        local_G.append(G_new)
        local_D.append(D_new)
        sizes.append(len(client_data))

    global_G = federated_average(local_G, sizes).to(device)
    global_D = federated_average(local_D, sizes).to(device)

    # === Train Classifier ===
    global_C.train()
    opt_C = optim.Adam(global_C.parameters(), lr=0.001)
    all_data, all_labels = [], []
    for client_data in client_partitions:
        loader = DataLoader(client_data, batch_size=64, shuffle=True)
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            z = torch.randn(xb.size(0), latent_dim).to(device)
            fake_data = global_G(z, yb).detach()
            combined_x = torch.cat([xb, fake_data], dim=0)
            combined_y = torch.cat([yb, yb], dim=0)
            all_data.append(combined_x)
            all_labels.append(combined_y)

    train_loader = DataLoader(TensorDataset(torch.cat(all_data), torch.cat(all_labels)), batch_size=128, shuffle=True)
    for xb, yb in train_loader:
        if xb.shape[0] < 2: continue
        xb, yb = xb.to(device), yb.to(device)
        preds = global_C(xb)
        loss = focal_loss(preds, yb)
        opt_C.zero_grad(); loss.backward(); opt_C.step()
    scheduler_C.step()

    # === Pseudo-labeling ===
    pseudo_data, pseudo_labels = get_confident_entropy_pseudo_labels(global_C, X_test_tensor)
    if len(pseudo_data) > 1:
        print(f"Adding {len(pseudo_data)} pseudo-labeled samples to training.")
        semi_loader = DataLoader(
            TensorDataset(torch.cat([X_train_tensor, pseudo_data]), torch.cat([y_train_tensor, pseudo_labels])),
            batch_size=128, shuffle=True
        )
        opt_C_pseudo = optim.Adam(global_C.parameters(), lr=0.001)
        for xb, yb in semi_loader:
            if xb.shape[0] < 2: continue
            xb = xb.clone().detach().to(device)
            yb = yb.clone().detach().to(device)
            preds = global_C(xb)
            loss = focal_loss(preds, yb)
            opt_C_pseudo.zero_grad(); loss.backward(); opt_C_pseudo.step()

# === Evaluation ===
global_C.eval()
with torch.no_grad():
    preds = global_C(X_test_tensor.to(device))
    y_pred = preds.argmax(dim=1).cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()

print("\n✅ Accuracy:", accuracy_score(y_true, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_true, y_pred))

# === Save Models ===
torch.save(global_G.state_dict(), "fgan2_generator_fm.pth")
torch.save(global_D.state_dict(), "fgan2_discriminator_fm.pth")
torch.save(global_C.state_dict(), "fgan2_classifier_fm.pth")



=== Epoch 1 ===


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:227: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Adding 1358 pseudo-labeled samples to training.

=== Epoch 2 ===
Adding 5516 pseudo-labeled samples to training.

=== Epoch 3 ===
Adding 4497 pseudo-labeled samples to training.

=== Epoch 4 ===
Adding 5623 pseudo-labeled samples to training.

=== Epoch 5 ===
Adding 6077 pseudo-labeled samples to training.

=== Epoch 6 ===
Adding 7014 pseudo-labeled samples to training.

=== Epoch 7 ===
Adding 6359 pseudo-labeled samples to training.

=== Epoch 8 ===
Adding 5932 pseudo-labeled samples to training.

=== Epoch 9 ===
Adding 5283 pseudo-labeled samples to training.

=== Epoch 10 ===
Adding 6542 pseudo-labeled samples to training.

=== Epoch 11 ===
Adding 6555 pseudo-labeled samples to training.

=== Epoch 12 ===
Adding 6514 pseudo-labeled samples to training.

=== Epoch 13 ===
Adding 4313 pseudo-labeled samples to training.

=== Epoch 14 ===
Adding 5666 pseudo-labeled samples to training.

=== Epoch 15 ===
Adding 5434 pseudo-labeled samples to training.

=== Epoch 16 ===
Adding 5169 pseudo

fgan2 with dec

In [ ]:
# === FGAN-II with DEC Pseudo-Labeling and Feature Matching Loss ===
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.cluster import KMeans
import joblib
import copy

# === Load and Preprocess Dataset ===
df = pd.read_csv("new_network_train.csv")
X = df.drop(columns=["ProtocolName"]).values
y = df["ProtocolName"].astype(np.int64).values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, "scaler_fgani.pkl")

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# === Constants ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_dim = X.shape[1]
num_classes = len(np.unique(y))
latent_dim = 100
NUM_CLIENTS = 5
label_smooth_real = 0.9

# === Partition Data ===
dataset = TensorDataset(X_train_tensor, y_train_tensor)
partition_sizes = [len(dataset) // NUM_CLIENTS + (1 if i < len(dataset) % NUM_CLIENTS else 0) for i in range(NUM_CLIENTS)]
client_partitions = random_split(dataset, partition_sizes)

# === Utility Functions ===
def one_hot(labels, num_classes):
    return torch.eye(num_classes, device=labels.device)[labels]

def get_pseudo_labels_via_dec(model, data, k=num_classes):
    model.eval()
    data = data.to(device)
    with torch.no_grad():
        features = model.net[:-1](data).cpu().numpy()
    kmeans = KMeans(n_clusters=k, random_state=0).fit(features)
    pseudo_data = data.cpu()
    pseudo_labels = torch.tensor(kmeans.labels_, dtype=torch.long)
    return pseudo_data, pseudo_labels

class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0):
        super().__init__()
        self.gamma = gamma
    def forward(self, input, target):
        logpt = F.log_softmax(input, dim=1)
        pt = torch.exp(logpt)
        loss = (1 - pt) ** self.gamma * logpt
        return F.nll_loss(loss, target)

# === Models ===
class Generator(nn.Module):
    def __init__(self, noise_dim, label_dim, output_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(noise_dim + label_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, output_dim),
        )
    def forward(self, noise, labels):
        labels = one_hot(labels, num_classes).to(noise.device)
        x = torch.cat([noise, labels], dim=1)
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, input_dim, label_dim):
        super().__init__()
        self.feature = nn.Sequential(
            nn.Linear(input_dim + label_dim, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    def forward(self, data, labels):
        labels = one_hot(labels, num_classes).to(data.device)
        x = torch.cat([data, labels], dim=1)
        features = self.feature(x)
        return self.classifier(features), features

class CNNClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes)
        )
    def forward(self, x):
        return self.net(x)

# === Federated Functions ===
def federated_average(models, sizes):
    avg_model = copy.deepcopy(models[0])
    total = sum(sizes)
    for key in avg_model.state_dict():
        weighted_sum = sum(model.state_dict()[key] * (size / total) for model, size in zip(models, sizes))
        avg_model.state_dict()[key].copy_(weighted_sum)
    return avg_model

# === Training Functions ===
def train_fgan2_client(client_data, global_G, global_D):
    G = copy.deepcopy(global_G).train()
    D = copy.deepcopy(global_D).train()
    loader = DataLoader(client_data, batch_size=64, shuffle=True)
    opt_D = optim.Adam(D.parameters(), lr=0.0001)
    opt_G = optim.Adam(G.parameters(), lr=0.0002)
    bce = nn.BCELoss()

    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        batch_size = xb.size(0)
        real_labels = torch.full((batch_size, 1), label_smooth_real).to(device)
        fake_labels = torch.zeros((batch_size, 1)).to(device)

        z = torch.randn(batch_size, latent_dim).to(device)
        fake_data = G(z, yb)
        D_real_out, _ = D(xb, yb)
        D_fake_out, _ = D(fake_data.detach(), yb)
        loss_D = bce(D_real_out, real_labels) + bce(D_fake_out, fake_labels)
        opt_D.zero_grad(); loss_D.backward(); opt_D.step()

        _, real_features = D(xb, yb)
        _, fake_features = D(fake_data, yb)
        loss_G = F.mse_loss(fake_features.mean(dim=0), real_features.mean(dim=0))
        opt_G.zero_grad(); loss_G.backward(); opt_G.step()

    return G, D

# === Training Loop ===
global_G = Generator(latent_dim, num_classes, input_dim).to(device)
global_D = Discriminator(input_dim, num_classes).to(device)
global_C = CNNClassifier(input_dim, num_classes).to(device)
scheduler_C = optim.lr_scheduler.StepLR(optim.Adam(global_C.parameters()), step_size=15, gamma=0.5)
focal_loss = FocalLoss()

EPOCHS = 30
for epoch in range(EPOCHS):
    print(f"\n=== Epoch {epoch + 1} ===")
    local_G, local_D, sizes = [], [], []

    for client_data in client_partitions:
        G_new, D_new = train_fgan2_client(client_data, global_G, global_D)
        local_G.append(G_new)
        local_D.append(D_new)
        sizes.append(len(client_data))

    global_G = federated_average(local_G, sizes).to(device)
    global_D = federated_average(local_D, sizes).to(device)

    global_C.train()
    opt_C = optim.Adam(global_C.parameters(), lr=0.001)
    all_data, all_labels = [], []
    for client_data in client_partitions:
        loader = DataLoader(client_data, batch_size=64, shuffle=True)
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            z = torch.randn(xb.size(0), latent_dim).to(device)
            fake_data = global_G(z, yb).detach()
            combined_x = torch.cat([xb, fake_data], dim=0)
            combined_y = torch.cat([yb, yb], dim=0)
            all_data.append(combined_x)
            all_labels.append(combined_y)

    train_loader = DataLoader(TensorDataset(torch.cat(all_data), torch.cat(all_labels)), batch_size=128, shuffle=True)
    for xb, yb in train_loader:
        if xb.shape[0] < 2: continue
        xb, yb = xb.to(device), yb.to(device)
        preds = global_C(xb)
        loss = focal_loss(preds, yb)
        opt_C.zero_grad(); loss.backward(); opt_C.step()
    scheduler_C.step()

    # === DEC Pseudo-labeling ===
    pseudo_data, pseudo_labels = get_pseudo_labels_via_dec(global_C, X_test_tensor)
    if len(pseudo_data) > 1:
        print(f"Adding {len(pseudo_data)} pseudo-labeled samples to training.")
        semi_loader = DataLoader(
            TensorDataset(torch.cat([X_train_tensor, pseudo_data]), torch.cat([y_train_tensor, pseudo_labels])),
            batch_size=128, shuffle=True
        )
        opt_C_pseudo = optim.Adam(global_C.parameters(), lr=0.001)
        for xb, yb in semi_loader:
            if xb.shape[0] < 2: continue
            xb = xb.clone().detach().to(device)
            yb = yb.clone().detach().to(device)
            preds = global_C(xb)
            loss = focal_loss(preds, yb)
            opt_C_pseudo.zero_grad(); loss.backward(); opt_C_pseudo.step()

# === Evaluation ===
global_C.eval()
with torch.no_grad():
    preds = global_C(X_test_tensor.to(device))
    y_pred = preds.argmax(dim=1).cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()

print("\n✅ Accuracy:", accuracy_score(y_true, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_true, y_pred))

# === Save Models ===
torch.save(global_G.state_dict(), "fgan2_generator_dec.pth")
torch.save(global_D.state_dict(), "fgan2_discriminator_dec.pth")
torch.save(global_C.state_dict(), "fgan2_classifier_dec.pth")


=== Epoch 1 ===


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:227: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Adding 26928 pseudo-labeled samples to training.

=== Epoch 2 ===
Adding 26928 pseudo-labeled samples to training.

=== Epoch 3 ===
Adding 26928 pseudo-labeled samples to training.

=== Epoch 4 ===
Adding 26928 pseudo-labeled samples to training.

=== Epoch 5 ===
Adding 26928 pseudo-labeled samples to training.

=== Epoch 6 ===
Adding 26928 pseudo-labeled samples to training.

=== Epoch 7 ===
Adding 26928 pseudo-labeled samples to training.

=== Epoch 8 ===
Adding 26928 pseudo-labeled samples to training.

=== Epoch 9 ===
Adding 26928 pseudo-labeled samples to training.

=== Epoch 10 ===
Adding 26928 pseudo-labeled samples to training.

=== Epoch 11 ===
Adding 26928 pseudo-labeled samples to training.

=== Epoch 12 ===
Adding 26928 pseudo-labeled samples to training.

=== Epoch 13 ===
Adding 26928 pseudo-labeled samples to training.

=== Epoch 14 ===
Adding 26928 pseudo-labeled samples to training.

=== Epoch 15 ===
Adding 26928 pseudo-labeled samples to training.

=== Epoch 16 ===
Add